<a href="https://colab.research.google.com/github/matkubita/StockAIFinanceRLAgent/blob/main/RLAgent_2_2_Train_DDPG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Setup

In [ ]:
!pip install swig
!pip install wrds
!pip install pyportfolioopt
!pip install git+https://github.com/AI4Finance-Foundation/FinRL.git

  Using cached pyportfolioopt-1.5.6-py3-none-any.whl.metadata (22 kB)
  Using cached ecos-2.0.14-cp312-cp312-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (8.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.1/222.1 kB 9.5 MB/s eta 0:00:00
  Cloning https://github.com/AI4Finance-Foundation/FinRL.git to /tmp/pip-req-build-qypt_se0
  Running command git clone --filter=blob:none --quiet https://github.com/AI4Finance-Foundation/FinRL.git /tmp/pip-req-build-qypt_se0
  Resolved https://github.com/AI4Finance-Foundation/FinRL.git to commit fee45af12ee0af490cd8e091514173b571dcd9ed
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Cloning https://github.com/AI4Finance-Foundation/ElegantRL.git to /tmp/pip-install-q_6rfo4q/elegantrl_9bd2d1f1e6704fe292e9b6a56ccfa336
  Running command git clone --filter=blob:none --quiet ht

In [ ]:
import os
import pandas as pd

from finrl.meta.env_stock_trading.env_stocktrading import StockTradingEnv
from finrl.agents.stablebaselines3.models import DRLAgent
from stable_baselines3.common.logger import configure
from finrl import config_tickers
from finrl.main import check_and_make_directories
from finrl.config import INDICATORS, TRAINED_MODEL_DIR, RESULTS_DIR

check_and_make_directories([TRAINED_MODEL_DIR])

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=

# Load the Data

In [ ]:
from google. colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ZMUM_FinRL/train_data.csv')

if "Unnamed: 0" in train.columns:
    train = train.drop(columns=["Unnamed: 0"])

train = train.sort_values(['date', 'tic']).reset_index(drop=True)

train.index = train['date'].factorize()[0]

         date    tic       close        high         low        open  \
0  2018-01-02   AAPL   40.341892   40.351262   39.639317   39.850091   
0  2018-01-02   AMZN   59.450500   59.500000   58.525501   58.599998   
0  2018-01-02  BRK-B  197.220001  198.869995  195.960007  198.869995   
0  2018-01-02  GOOGL   53.258095   53.395556   52.256167   52.256167   
0  2018-01-02    JPM   86.730354   86.786594   85.814444   86.473257   
0  2018-01-02   META  180.161514  180.320408  176.318365  176.447453   
0  2018-01-02   MSFT   79.050369   79.381471   78.636496   79.215920   
0  2018-01-02   NVDA    4.928533    4.932242    4.808627    4.840272   
0  2018-01-02   TSLA   21.368668   21.474001   20.733334   20.799999   
0  2018-01-02    WMT   28.914410   29.266343   28.893879   29.122635   
1  2018-01-03   AAPL   40.334866   40.878192   40.271636   40.405124   
1  2018-01-03   AMZN   60.209999   60.274502   59.415001   59.415001   
1  2018-01-03  BRK-B  199.789993  200.000000  197.000000  197.00

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


# Setup environemnt for training

In [ ]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(INDICATORS)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")

Stock Dimension: 10, State Space: 101


In [ ]:
buy_cost_list = sell_cost_list = [0.001] * stock_dimension
num_stock_shares = [0] * stock_dimension

env_kwargs = {
    "hmax": 100,
    "initial_amount": 1000000,
    "num_stock_shares": num_stock_shares,
    "buy_cost_pct": buy_cost_list,
    "sell_cost_pct": sell_cost_list,
    "state_space": state_space,
    "stock_dim": stock_dimension,
    "tech_indicator_list": INDICATORS,
    "action_space": stock_dimension,
    "reward_scaling": 1e-4
}


e_train_gym = StockTradingEnv(df = train, **env_kwargs)

In [ ]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))

<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [ ]:
agent = DRLAgent (env = env_train)

# Set the corresponding values to 'True
if_using_a2c = False
if_using_ddpg = True
if_using_ppo = False
if_using_td3 = False
if_using_sac = False

# Training agent - DDPG model

In [29]:
model_ddpg = agent.get_model("ddpg")

if if_using_ddpg:
    model_ddpg.learn(total_timesteps=50000, tb_log_name='ddpg')
    trained_ddpg = model_ddpg

---------------------------------
| time/              |          |
|    episodes        | 4        |
|    fps             | 38       |
|    time_elapsed    | 131      |
|    total_timesteps | 5036     |
| train/             |          |
|    actor_loss      | 126      |
|    critic_loss     | 2.93e+04 |
|    learning_rate   | 0.001    |
|    n_updates       | 5157     |
---------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


day: 1258, episode: 30
begin_total_asset: 1000000.00
end_total_asset: 1354854.67
total_reward: 354854.67
total_cost: 998.97
total_trades: 6290
Sharpe: 0.355


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| time/              |          |
|    episodes        | 8        |
|    fps             | 38       |
|    time_elapsed    | 264      |
|    total_timesteps | 10072    |
| train/             |          |
|    actor_loss      | -2.13    |
|    critic_loss     | 607      |
|    learning_rate   | 0.001    |
|    n_updates       | 10193    |
---------------------------------


...

/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


---------------------------------
| time/              |          |
|    episodes        | 32       |
|    fps             | 37       |
|    time_elapsed    | 1079     |
|    total_timesteps | 40288    |
| train/             |          |
|    actor_loss      | 0.343    |
|    critic_loss     | 1.04     |
|    learning_rate   | 0.001    |
|    n_updates       | 40409    |
---------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


day: 1258, episode: 60
begin_total_asset: 1000000.00
end_total_asset: 1354854.67
total_reward: 354854.67
total_cost: 998.97
total_trades: 6290
Sharpe: 0.355
---------------------------------
| time/              |          |
|    episodes        | 36       |
|    fps             | 37       |
|    time_elapsed    | 1221     |
|    total_timesteps | 45324    |
| train/             |          |
|    actor_loss      | -1.09    |
|    critic_loss     | 0.745    |
|    learning_rate   | 0.001    |
|    n_updates       | 45445    |
---------------------------------


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


In [32]:
# Definicja ścieżki docelowej na Google Drive
drive_save_path = "/content/drive/MyDrive/ZMUM_FinRL/agent_ddpg"

# Bezpośredni zapis na dysku
if if_using_ddpg:
    trained_ddpg.save(drive_save_path)
    print(f"✅ Model został zapisany bezpośrednio w: {drive_save_path}.zip")

✅ Model został zapisany bezpośrednio w: /content/drive/MyDrive/ZMUM_FinRL/agent_ddpg.zip
